In [18]:
import nb_setup
import importlib

importlib.reload(nb_setup)
nb_setup.init()

Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [19]:
import csv
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

df = pd.read_csv("data/BTC.csv")

import models.backtesting.backtesting as backtesting
import models.backtesting.BollingerBandsReEntryStrategy as bollingerBandsReEntryStrategy

importlib.reload(bollingerBandsReEntryStrategy)
importlib.reload(backtesting)
BackTesting = backtesting.BackTesting
EXPECTED_DTYPES = backtesting.EXPECTED_DTYPES
# importlib.reload(BackTesting)
# importlib.reload(EXPECTED_DTYPES)

df["datetime"] = pd.to_datetime(df["datetime"])
df = df.astype(EXPECTED_DTYPES)

df.shape


(2881, 13)

In [20]:
# Create a strategy
bt = BackTesting()
bt.run_backtesting(
    df,
    strategy=bollingerBandsReEntryStrategy.BollingerBandsReEntryStrategy(
        initial_capital=200,
        buy_equity=10,
        leverage=1,
    ),
    # play_mode=True,
)

Validating data...
Data validated successfully.
Start Running backtesting...
Data shape: (2881, 13)
Strategy: <models.backtesting.BollingerBandsReEntryStrategy.BollingerBandsReEntryStrategy object at 0x12f889b50>
Processing row 0...
Portfolio Value:  (1, 6) : 200
Processing row 1...
shape(self.data):  (2, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (2, 6) : 200
Processing row 2...
shape(self.data):  (3, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (3, 6) : 200
Processing row 3...
shape(self.data):  (4, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (4, 6) : 200
Processing row 4...
shape(self.data):  (5, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (5, 6) : 200
Processing row 5...
shape(self.data):  (6, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (6, 6) : 200
Processing row 6...
shape(self.data):  (7, 13) states:  nan nan
No valid trade condition met.
Portfolio Value: 

/Users/falcon/Developments/act/backtesting-playground/trading/models/backtesting/BollingerBandsReEntryStrategy.py:140: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/falcon/Developments/act/backtesting-playground/trading/models/backtesting/portfolio.py:234: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



shape(self.data):  (29, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (29, 6) : 200
Processing row 29...
shape(self.data):  (30, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (30, 6) : 200
Processing row 30...
shape(self.data):  (31, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (31, 6) : 200
Processing row 31...
shape(self.data):  (32, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (32, 6) : 200
Processing row 32...
shape(self.data):  (33, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (33, 6) : 200
Processing row 33...
shape(self.data):  (34, 13) states:  BollingerBandsState.WITHIN_BANDS 

/Users/falcon/Developments/act/backtesting-playground/trading/models/backtesting/BollingerBandsReEntryStrategy.py:263: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Portfolio Value:  (148, 6) : 199.8464461711955262385491171
Processing row 148...
shape(self.data):  (149, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (149, 6) : 199.8666831446093955169697063
Processing row 149...
shape(self.data):  (150, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (150, 6) : 199.8768788716728716419754994
Processing row 150...
shape(self.data):  (151, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (151, 6) : 199.8961889608082430908501074
Processing row 151...
shape(self.data):  (152, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (152, 6) : 199.8974248065129068635780823
Processing row 152...
shape(self.data):  (153, 13) states:  BollingerBandsState.W

In [24]:
bt.strategy.portfolio.holdings

{'BTC': [Symbol: BTC, Entry Price: 62340, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001604106512672441450112287456,
  Symbol: BTC, Entry Price: 62958, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001588360494297785825470948887,
  Symbol: BTC, Entry Price: 62037, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001611941260860454245047310476,
  Symbol: BTC, Entry Price: 61010, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001639075561383379773807572529,
  Symbol: BTC, Entry Price: 63438, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001576342255430499069958069296,
  Symbol: BTC, Entry Price: 64644, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001546933976857867706206299115]}

In [25]:
# display(bt.strategy.history)
# display(bt.strategy.portfolio.portfolio_history)

In [26]:
from settings import DATA_DIR


name = "200_10-1x_BTC_BollingerBandsReEntryStrategy_3std"

bt.strategy.history.to_csv(DATA_DIR / f"./{name}-history.csv")
bt.strategy.portfolio.portfolio_history.to_csv(
    DATA_DIR / f"./{name}-portfolio-history.csv"
)

In [27]:
bt.strategy.plot_history()

In [28]:
bt.strategy.plot_performance()

In [29]:
bt.strategy.portfolio.info()

Start Time:  2024-03-23 00:00:00
End Time:  2024-04-22 00:00:00
Cash:  140.2212646639791036599374262
Holdings: 
{'BTC': [Symbol: BTC, Entry Price: 62340, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001604106512672441450112287456,
         Symbol: BTC, Entry Price: 62958, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001588360494297785825470948887,
         Symbol: BTC, Entry Price: 62037, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001611941260860454245047310476,
         Symbol: BTC, Entry Price: 61010, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001639075561383379773807572529,
         Symbol: BTC, Entry Price: 63438, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001576342255430499069958069296,
         Symbol: BTC, Entry Price: 64644, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001546933976857867706206299115]}
Evaluation:  202.5018293403660606423666818
PNL:  2.5018293403660606423666818
PNL Percentage:  1.250914670183030321183340900
Max DrawDown 